# 40 · Router Suite Workflow (Env → Extract → Manifest → Summarize)

End-to-end router analysis using *textual* exports only — no live capture in this lab.

In [ ]:
# Prepare synthetic router export folder with minimal files
from pathlib import Path, json
base = Path.cwd() / "lab_workspace" / "router_exports"
base.mkdir(parents=True, exist_ok=True)

(base / "portforwards.txt").write_text("tcp 8443 10.0.0.10 8443\n")
(base / "ddns.conf").write_text("provider=example; host=home.example.com\n")
(base / "eventlog.json").write_text(json.dumps([{"ts":"2025-01-01T00:10:00Z","msg":"WAN up"}], indent=2))

sorted(p.name for p in base.iterdir())

In [ ]:
# Run router extract via CLI if available, otherwise sketch expected outputs
import shutil, subprocess
from pathlib import Path

WORKSPACE = Path.cwd() / "lab_workspace"
CASE = "lab_demo_case"
if shutil.which("forensic-cli"):
    subprocess.run(["forensic-cli","--workspace",str(WORKSPACE),
                    "router","env","init","--case",CASE], check=False)
    subprocess.run(["forensic-cli","--workspace",str(WORKSPACE),
                    "router","extract","ui","--case",CASE,
                    "--param",f"input={str(base)}"], check=False)
    subprocess.run(["forensic-cli","--workspace",str(WORKSPACE),
                    "router","manifest","write","--case",CASE], check=False)
    subprocess.run(["forensic-cli","--workspace",str(WORKSPACE),
                    "router","summarize","--case",CASE], check=False)
else:
    print("forensic-cli not found — demonstrating expected artifact paths only.")
    case_dir = WORKSPACE / "cases" / CASE
    print("Expected outputs under:", case_dir / "router")